<a href="https://colab.research.google.com/github/ML-Challenge/week5-preprocessing-and-tunning/blob/master/L2.Hyperparameter%20Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

Building powerful machine learning models depends heavily on the set of hyperparameters used. But with increasingly complex models with lots of options, how do we efficiently find the best settings for our particular problem? In this lesson we will get practical experience in using some common methodologies for automated hyperparameter tuning in Python using Scikit Learn. These include Grid Search, Random Search & advanced optimization methodologies including Bayesian & Genetic algorithms. We will use a dataset predicting credit card defaults as we build skills to dramatically increase the efficiency and effectiveness of our machine learning model building.

# Setup

In [ ]:
# Download lesson datasets
# Required if you're using Google Colab
#!wget "https://github.com/ML-Challenge/week5-preprocessing-and-tunning/raw/master/datasets.zip"
#!unzip -o datasets.zip

In [ ]:
# Import utils
# We'll be using this module throughout the lesson
import utils

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# and setting the size of all plots.
plt.rcParams['figure.figsize'] = [20, 10]

from sklearn.model_selection import train_test_split

# Hyperparameters and Parameters

In this introductory section, we will learn the difference between hyperparameters and parameters. We will practice extracting and analyzing parameters, setting hyperparameter values for several popular machine learning algorithms. Along the way we will learn some best practice tips & tricks for choosing which hyperparameters to tune and what values to set & build learning curves to analyze our hyperparameter choices.

## Introduction & 'Parameters'

### Parameters in Logistic Regression

Now that we have had a chance to explore what a parameter is, let us apply this knowledge. It is important to be able to review any new algorithm and identify which elements are parameters and hyperparameters.

Which of the following is a parameter for the Scikit Learn logistic regression model?

**Possible Answers**

1. `n_jobs`
2. `coef_`
3. `class_weight`
4. `LogisticRegression()`

In [ ]:
# Pass 1,2,3 or 4 as argument
utils.which_is_param()

### Extracting a Logistic Regression parameter

We are now going to practice extracting an important parameter of the logistic regression model. The logistic regression has a few other parameters we will not explore here, but they can be reviewed in the **scikit-learn.org** documentation for the `LogisticRegression()` module under 'Attributes'.

This parameter is important for understanding the direction and magnitude of the effect the variables have on the target.

In this example we will extract the coefficient parameter (found in the `coef_` attribute), zip it up with the original column names, and see which variables had the largest positive effect on the target variable.

In [ ]:
utils.credit_card.head()

In [ ]:
X = utils.credit_card.drop('default payment next month', axis=1)
y = utils.credit_card['default payment next month']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [ ]:
X_train.var()

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_clf = LogisticRegression(solver='liblinear', random_state=42)

In [ ]:
log_reg_clf.fit(X_train, y_train)

In [ ]:
# Create a list of original variable names from the training DataFrame
original_variables = X_train.columns

In [ ]:
# Extract the coefficients of the logistic regression estimator
model_coefficients = log_reg_clf.coef_[0]

In [ ]:
# Create a dataframe of the variables and coefficients & print it out
coefficient_df = pd.DataFrame({"Variable" : original_variables, "Coefficient": model_coefficients})
coefficient_df

In [ ]:
# Print out the top 3 positive variables
top_three_df = coefficient_df.sort_values(by="Coefficient", axis=0, ascending=False)[0:3]
top_three_df

We have succesfully extracted and reviewed a very important parameter for the Logistic Regression Model. The coefficients of the model allow us to see which variables are having a larger or smaller impact on the outcome. Additionally the sign lets us know if it is a positive or negative relationship.

### Extracting a Random Forest parameter

We will now translate the work previously undertaken on the logistic regression model to a random forest model. A parameter of this model is, for a given tree, how it decided to split at each level.

This analysis is not as useful as the coefficients of logistic regression as we will be unlikely to ever explore every split and every tree in a random forest model. However, it is a very useful exercise to peak under the hood at what the model is doing.

In this example we will extract a single tree from our random forest model, visualize it and programmatically extract one of the splits.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(max_depth=4, random_state=42)

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
# Extract the 7th (index 6) tree from the random forest
chosen_tree = rf_clf.estimators_[6]

In [ ]:
# Visualize the graph using Graphviz

#from sklearn.tree import export_graphviz

#export_graphviz(chosen_tree, out_file='assets/tree.dot', 
#                feature_names = original_variables, class_names = True,
#                rounded = True, proportion = False, precision = 2, filled = True)

# Convert to png using system command 
#from subprocess import call
#call(['dot', '-Tpng', 'assets/tree.dot', '-o', 'assets/tree.png', '-Gdpi=600'])

# Display in jupyter notebook
#from IPython.display import Image
#Image(filename = 'assets/tree.png')

In [ ]:
from sklearn.tree import plot_tree

plot_tree(chosen_tree,feature_names = original_variables, class_names = True,
                rounded = True, proportion = False, precision = 2, filled = True, fontsize=10)

## Introducing Hyperparameters

### Hyperparameters in Random Forests

As we saw, there are many different hyperparameters available in a Random Forest model using Scikit Learn. Let's try to differentiate between a hyperparameter and a parameter, and easily check whether something is a hyperparameter.

We can create a random forest estimator from the imported Scikit Learn package. Then print this estimator out to see the hyperparameters and their values.

Which of the following is a hyperparameter for the Scikit Learn random forest model?

In [ ]:
rf_clf

**Possible Answers**

1. `oob_score`
2. `classes_`
3. `trees`
4. `random_level`

In [ ]:
# Pass 1,2,3 or 4 as argument
utils.which_is_hyperparam()

### Exploring Random Forest Hyperparameters

Understanding what hyperparameters are available and the impact of different hyperparameters is a core skill for any data scientist. As models become more complex, there are many different settings we can set, but only some will have a large impact on our model.

We will now assess an existing random forest model (it has some bad choices for hyperparameters!) and then make better choices for a new random forest model and assess its performance.

In [ ]:
rf_clf_bad = RandomForestClassifier(n_estimators=5, random_state=42)
rf_clf_bad.fit(X_train, y_train)
rf_bad_predictions = rf_clf_bad.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Get confusion matrix & accuracy for the bad rf_model
print(f'Confusion Matrix: \n\n {confusion_matrix(y_test, rf_bad_predictions)} \n Accuracy Score: \n\n {accuracy_score(y_test, rf_bad_predictions)}')

In [ ]:
# Create a new random forest classifier with better hyperparamaters
rf_clf_new = RandomForestClassifier(n_estimators=500, random_state=42)

In [ ]:
# Fit this to the data and obtain predictions
rf_new_predictions = rf_clf_new.fit(X_train, y_train).predict(X_test)

In [ ]:
# Get confusion matrix & accuracy for the new rf_model
print(f'Confusion Matrix: \n\n {confusion_matrix(y_test, rf_new_predictions)} \n Accuracy Score: \n\n {accuracy_score(y_test, rf_new_predictions)}')

We got a nice 3% accuracy boost just from changing the `n_estimators`. We have had our first taste of hyperparameter tuning for a random forest model.

### Hyperparameters of KNN

The k-nearest-neighbors algorithm is not as popular as it used to be, but can still be an excellent choice for data that has groups of data that behave similarly. Could this be the case for our credit card users?

In this case we will try out several different values for one of the core hyperparameters for the knn algorithm and compare performance.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Build a knn estimator for each value of n_neighbours
knn_5 = KNeighborsClassifier(n_neighbors=5)
knn_10 = KNeighborsClassifier(n_neighbors=10)
knn_20 = KNeighborsClassifier(n_neighbors=20)

In [ ]:
# Fit each to the training data & produce predictions
knn_5_predictions = knn_5.fit(X_train, y_train).predict(X_test)
knn_10_predictions = knn_10.fit(X_train, y_train).predict(X_test)
knn_20_predictions = knn_20.fit(X_train, y_train).predict(X_test)

In [ ]:
# Get an accuracy score for each of the models
knn_5_accuracy = accuracy_score(y_test, knn_5_predictions)
knn_10_accuracy = accuracy_score(y_test, knn_10_predictions)
knn_20_accuracy = accuracy_score(y_test, knn_20_predictions)

print(f"The accuracy of 5, 10, 20 neighbours was {knn_5_accuracy}, {knn_10_accuracy}, {knn_20_accuracy}")

We succesfully tested 3 different options for 1 hyperparameter, but it was pretty exhausting. Next, we will try to find a way to make this easier.

## Setting & Analyzing Hyperparameter Values

### Automating Hyperparameter Choice

Finding the best hyperparameter of interest without writing hundreds of lines of code for hundreds of models is an important efficiency gain that will greatly assist our future machine learning model building.

An important hyperparameter for the GBM algorithm is the learning rate. But which learning rate is best for this problem? By writing a loop to search through a number of possibilities, collating these and viewing them we can find the best one.

Possible learning rates to try include 0.001, 0.01, 0.05, 0.1, 0.2 and 0.5

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Set the learning rates & results storage
learning_rates = [0.001, 0.01, 0.05, 0.1, 0.2, 0.5]
results_list = []

In [ ]:
# Create the for loop to evaluate model predictions for each learning rate
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(learning_rate=learning_rate, random_state=42)
    predictions = model.fit(X_train, y_train).predict(X_test)
    # Save the learning rate and accuracy score
    results_list.append([learning_rate, accuracy_score(y_test, predictions)])

In [ ]:
# Gather everything into a DataFrame
results_df = pd.DataFrame(results_list, columns=['learning_rate', 'accuracy'])
print(results_df)

We efficiently tested a few different values for a single hyperparameter and can easily see which learning rate value was the best. Here, it seems that a learning rate of 0.05 yields the best accuracy.

### Building Learning Curves

If we want to test many different values for a single hyperparameter it can be difficult to easily view that in the form of a DataFrame. Previously we learned about a nice trick to analyze this. A graph called a 'learning curve' can nicely demonstrate the effect of increasing or decreasing a particular hyperparameter on the final result.

Instead of testing only a few values for the learning rate, we will test many to easily see the effect of this hyperparameter across a large range of values. A useful function from NumPy is `np.linspace(start, end, num)` which allows us to create a number of values (`num`) evenly spread within an interval (`start`, `end`) that we specify.

In [ ]:
# Set the learning rates & accuracies list
learn_rates = np.linspace(0.01, 2, num=30)
accuracies = []

In [ ]:
# Create the for loop
for learn_rate in learn_rates:
    # Create the model, predictions & save the accuracies as before
    model = GradientBoostingClassifier(learning_rate=learn_rate, random_state=42)
    predictions = model.fit(X_train, y_train).predict(X_test)
    accuracies.append(accuracy_score(y_test, predictions))

In [ ]:
# Plot results    
plt.plot(learn_rates, accuracies)
plt.gca().set(xlabel='learning_rate', ylabel='Accuracy', title='Accuracy for different learning_rates')
plt.show() 

We can see that for low values, we get a pretty good accuracy. However once the learning rate pushes much above 1.5, the accuracy starts to drop. We have learned and practiced a useful skill for visualizing large amounts of results for a single hyperparameter.

# Grid search

This part of the lesson introduces a popular automated hyperparameter tuning methodology called Grid Search. We will learn what it is, how it works and practice undertaking a Grid Search using Scikit Learn. We will then learn how to analyze the output of a Grid Search & gain practical experience doing this.

## Introducing Grid Search

### Build Grid Search functions

In data science it is a great idea to try building algorithms, models and processes 'from scratch' so we can really understand what is happening at a deeper level. Of course there are great packages and libraries for this work (and we will get to that very soon!) but building from scratch will give us a great edge in our data science work.

In this example, we will create a function to take in 2 hyperparameters, build models and return results. We will use this function in a future example.

In [ ]:
# Create the function
def gbm_grid_search(learn_rate, max_depth, random_state=42):

    # Create the model
    model = GradientBoostingClassifier(learning_rate=learn_rate, max_depth=max_depth, random_state=random_state)
    
    # Use the model to make predictions
    predictions = model.fit(X_train, y_train).predict(X_test)
    
    # Return the hyperparameters and score
    return([learn_rate, max_depth, accuracy_score(y_test, predictions)])

### Iteratively tune multiple hyperparameters

In this example, we will build on the function we previously created to take in 2 hyperparameters, build a model and return the results. We will now use that to loop through some values and then extend this function and loop with another hyperparameter.

In [ ]:
# Create the relevant lists
results_list = []
learn_rate_list = [0.01, 0.1, 0.5]
max_depth_list = [2,4,6]

In [ ]:
# Create the for loop
for learn_rate in learn_rate_list:
    for max_depth in max_depth_list:
        results_list.append(gbm_grid_search(learn_rate,max_depth))

In [ ]:
# Print the results
results_list

In [ ]:
# Extend the function input
def gbm_grid_search_extended(learn_rate, max_depth, subsample, random_state=42):

    # Extend the model creation section
    model = GradientBoostingClassifier(learning_rate=learn_rate, max_depth=max_depth, subsample=subsample, random_state=42)
    
    predictions = model.fit(X_train, y_train).predict(X_test)
    
    # Extend the return part
    return([learn_rate, max_depth, subsample, accuracy_score(y_test, predictions)])

In [ ]:
results_list = []

# Create the new list to test
subsample_list = [0.4, 0.6]

In [ ]:
for learn_rate in learn_rate_list:
    for max_depth in max_depth_list:
        # Extend the for loop
        for subsample in subsample_list:
            # Extend the results to include the new hyperparameter
            results_list.append(gbm_grid_search_extended(learn_rate, max_depth, subsample))

In [ ]:
# Print results
results_list

We have effectively built our own grid search! We went from 2 to 3 hyperparameters and can see how we could extend that to even more values and hyperparameters. That was a lot of effort though. Be warned - we are now entering a world that can get very computationally expensive very fast!

### How Many Models?

Adding more hyperparameters or values, we increase the amount of models created but the increase is not linear, it is proportional to how many values and hyperparameters we already have.

How many models would be created when running a grid search over the following hyperparameters and values for a GBM algorithm?

* learning_rate = [0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1, 2]
* max_depth = [4,6,8,10,12,14,16,18, 20]
* subsample = [0.4, 0.6, 0.7, 0.8, 0.9]
* max_features = ['auto', 'sqrt', 'log2']

**Possible Answers**

1. 26
2. 9 of one model, 9 of another
3. 1 large model
4. 1215

In [ ]:
# Enter 1, 2, 3 or 4 as the answer
utils.how_many_models()

## Grid Search with Scikit Learn

### GridSearchCV inputs

Let's test our knowledge of GridSeachCV inputs by answering the question below.

Three GridSearchCV objects are available below. Note that there is no data available to fit these models. Instead, answer by looking at their construct.

```
Model #1:
 GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'max_depth': [2, 4, 8, 15], 'max_features': ['auto', 'sqrt']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0) 


Model #2:
 GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid={'n_neighbors': [5, 10, 20], 'algorithm': ['ball_tree', 'brute']},
       pre_dispatch='2*n_jobs', refit=False, return_train_score='warn',
       scoring='accuracy', verbose=0) 


Model #3:
 GridSearchCV(cv=7, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'number_attempts': [2, 4, 6], 'max_depth': [3, 6, 9, 12]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)
```

Which of these GridSearchCV objects would not work when we try to fit it?

**Possible Answers**

1. `Model #1` would not work when we try to fit it.
2. `Model #2` would not work when we try to fit it.
3. `Model #3` would not work when we try to fit it.
4. None - they will all work when we try to fit them.

In [ ]:
# Enter 1, 2, 3 or 4 as the answer
utils.which_grid_search()

### GridSearchCV with Scikit Learn

The `GridSearchCV` module from Scikit Learn provides many useful features to assist with efficiently undertaking a grid search. We will now put it into practice by creating a GridSearchCV object with certain parameters.

The desired options are:

* A Random Forest Estimator, with the split criterion as 'entropy'
* 5-fold cross validation
* The hyperparameters `max_depth` (2, 4, 8, 15) and `max_features` ('auto' vs 'sqrt')
* Use `roc_auc` to score the models
* Use 4 cores for processing in parallel
* Ensure we refit the best model and return training scores

In [ ]:
# Create a Random Forest Classifier with specified criterion
rf_class = RandomForestClassifier(criterion='entropy', random_state=42)

In [ ]:
# Create the parameter grid
param_grid = {'max_depth': [2, 4, 8, 15], 'max_features': ['auto', 'sqrt']}

In [ ]:
# Create a GridSearchCV object
from sklearn.model_selection import GridSearchCV

grid_rf_class=GridSearchCV(
    estimator=rf_class,
    param_grid=param_grid,
    scoring='roc_auc',
    n_jobs=4,
    cv=5,
    refit=True, 
    return_train_score=True)

grid_rf_class

## Understanding a grid search output

### Exploring the grid search results

We will now explore the `cv_results_` property of the GridSearchCV object defined above. This is a dictionary that we can read into a pandas DataFrame and contains a lot of useful information about the grid search we just undertook.

A reminder of the different column types in this property:

* `time_` columns
* `param_` columns (one for each hyperparameter) and **the** singular `params` column (with all hyperparameter settings)
* a `train_score` column for each cv fold including the `mean_train_score` and `std_train_score` columns
* a `test_score` column for each cv fold including the `mean_test_score` and `std_test_score` columns
* a `rank_test_score` column with a number from 1 to n (number of iterations) ranking the rows based on their `mean_test_score`

In [ ]:
# First fit the GridSearchCV
grid_rf_class.fit(X_train, y_train)

In [ ]:
# Read the cv_results property into a dataframe & print it out
cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
cv_results_df

In [ ]:
# Extract and print the column with a dictionary of hyperparameters used
column = cv_results_df.loc[:, ["params"]]
column

In [ ]:
# Extract and print the row that had the best mean test score
best_row = cv_results_df[cv_results_df["rank_test_score"] == 1]
best_row

### Analyzing the best results

At the end of the day, we primarily care about the best performing 'square' in a grid search. Luckily Scikit Learn's `gridSearchCV` objects have a number of parameters that provide key information on just the best square (or row in `cv_results_`).

Three properties we will explore are:

* `best_score_` – The score (here ROC_AUC) from the best-performing square.
* `best_index_` – The index of the row in `cv_results_` containing information on the best-performing square.
* `best_params_` – A dictionary of the parameters that gave the best score, for example `'max_depth': 10`

In [ ]:
# Print out the ROC_AUC score from the best-performing square
best_score = grid_rf_class.best_score_
best_score

In [ ]:
# Create a variable from the row related to the best-performing square
cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
best_row = cv_results_df.loc[[grid_rf_class.best_index_]]
best_row

In [ ]:
# Get the max_depth parameter from the best-performing square
best_max_depth = grid_rf_class.best_params_["max_depth"]
best_max_depth

Being able to quickly find and prioritize the huge volume of information given back from machine learning modeling output is a great skill. Here we had great practice doing that with `cv_results_` by quickly isolating the key information on the best performing square. This will be very important when our grids grow from 12 squares to many more!

### Using the best results

While it is interesting to analyze the results of our grid search, our final goal is practical in nature; we want to make predictions on our test set using our estimator object.

We can access this object through the `best_estimator_` property of our grid search object.

In this example, we will take a look inside the `best_estimator_` property and then use this to make predictions on our test set for credit card defaults and generate a variety of scores. Remember that we need to use `predict_proba` rather than `predict` since we need probability values rather than class labels for our roc_auc score. We use a slice `[:,1]` to get probabilities of the positive class.

In [ ]:
# See what type of object the best_estimator_ property is
type(grid_rf_class.best_estimator_)

In [ ]:
# Create an array of predictions directly using the best_estimator_ property
predictions = grid_rf_class.best_estimator_.predict(X_test)

In [ ]:
# Take a look to confirm it worked, this should be an array of 1's and 0's
predictions[0:5]

In [ ]:
# Now create a confusion matrix 
print("Confusion Matrix \n", confusion_matrix(y_test, predictions))

In [ ]:
from sklearn.metrics import roc_auc_score

# Get the ROC-AUC score
predictions_proba = grid_rf_class.best_estimator_.predict_proba(X_test)[:,1]
print("ROC-AUC Score \n", roc_auc_score(y_test, predictions_proba))

The `.best_estimator_` property is a really powerful property to understand for streamlining our machine learning model building process. We now can run a grid search and seamlessly use the best model from that search to make predictions.

# Random Search

Another popular automated hyperparameter tuning methodology is called Random Search. We will learn what it is, how it works and importantly how it differs from grid search. We will learn some advantages and disadvantages of this method and when to choose this method compared to Grid Search. We will practice undertaking a Random Search with Scikit Learn as well as visualizing & interpreting the output.

## Introducing Random Search

### Randomly Sample Hyperparameters

To undertake a random search, we firstly need to undertake a random sampling of our hyperparameter space.

In this example, we will firstly create some lists of hyperparameters that can be zipped up to a list of lists. Then we will randomly sample hyperparameter combinations preparation for running a random search.

We will use just the hyperparameters `learning_rate` and `min_samples_leaf` of the GBM algorithm to keep the example illustrative and not overly complicated.

In [ ]:
# Create a list of values for the learning_rate hyperparameter
learn_rate_list = list(np.linspace(0.01,1.5,200))

In [ ]:
# Create a list of values for the min_samples_leaf hyperparameter
min_samples_list = list(range(10,41))

In [ ]:
# Combination list
from itertools import product
combinations_list = [list(x) for x in product(learn_rate_list, min_samples_list)]

In [ ]:
# Sample hyperparameter combinations for a random search.
random_combinations_index = np.random.choice(range(0, len(combinations_list)), 250, replace=False)
combinations_random_chosen = [combinations_list[x] for x in random_combinations_index]

In [ ]:
# Print the result
combinations_random_chosen

We generated some hyperparameter combinations and randomly sampled in that space. The output was not too nice though, in the next example we will use a much more efficient method for this. In a future example we will also make this output look much nicer!

### Randomly Search with Random Forest

To solidify our knowledge of random sampling, let's try a similar exercise but using different hyperparameters and a different algorithm.

As before, we create some lists of hyperparameters that can be zipped up to a list of lists. We will use the hyperparameters `criterion`, `max_depth` and `max_features` of the random forest algorithm. Then we will randomly sample hyperparameter combinations in preparation for running a random search.

We will use a slightly different package for sampling in this task, `random.sample()`.

In [ ]:
# Create lists for criterion and max_features
criterion_list = ["gini", "entropy"]
max_feature_list = ["auto", "sqrt", "log2", None]

In [ ]:
# Create a list of values for the max_depth hyperparameter
max_depth_list = list(range(3,56))

In [ ]:
# Combination list
combinations_list = [list(x) for x in product(criterion_list, max_feature_list, max_depth_list)]

In [ ]:
import random

# Sample hyperparameter combinations for a random search
combinations_random_chosen = random.sample(combinations_list, 150)

In [ ]:
# Print the result
combinations_random_chosen

### Visualizing a Random Search

Visualizing the search space of random search allows us to easily see the coverage of this technique and therefore allows us to see the effect of our sampling on the search space.

In this example we will use several different samples of hyperparameter combinations and produce visualizations of the search space.

In [ ]:
def sample_hyperparameters(n_samples):
    global combinations_random_chosen

    if n_samples == len(combinations_list):
        combinations_random_chosen = combinations_list
        return

    combinations_random_chosen = []
    random_combinations_index = np.random.choice(range(0, len(combinations_list)), n_samples, replace=False)
    combinations_random_chosen = [combinations_list[x] for x in random_combinations_index]
    return

In [ ]:
def visualize_search():
    rand_y, rand_x = [x[0] for x in combinations_random_chosen], [x[1] for x in combinations_random_chosen]

    # Plot all together
    plt.clf() 
    plt.scatter(rand_y, rand_x, c=['blue']*len(combinations_random_chosen))
    plt.gca().set(xlabel='learn_rate', ylabel='min_samples_leaf', title='Random Search Hyperparameters')
    plt.gca().set_xlim([0.01, 1.5])
    plt.gca().set_ylim([10, 29])
    plt.show()

In [ ]:
# Create a list of values for the learning_rate hyperparameter
learn_rate_list = list(np.linspace(0.01,1.5,200))

In [ ]:
# Create a list of values for the min_samples_leaf hyperparameter
min_samples_list = list(range(10,41))

In [ ]:
# Combination list
combinations_list = [list(x) for x in product(learn_rate_list, min_samples_list)]

In [ ]:
# Confirm how hyperparameter combinations & print
number_combs = len(combinations_list)
number_combs

In [ ]:
# Sample and visualise combinations
for x in [250, 1500, 5000]:
    sample_hyperparameters(x)
    visualize_search()

Notice how the bigger our sample space of a random search the more it looks like a grid search? In a later example we will look closer at comparing these two methods side by side.

## Random Search in Scikit Learn

### The RandomizedSeachCV Class

Just like the `GridSearchCV` library from Scikit Learn, `RandomizedSearchCV` provides many useful features to assist with efficiently undertaking a random search. We're going to create a `RandomizedSearchCV` object, making the small adjustment needed from the `GridSearchCV` object.

The desired options are:

* A default Gradient Boosting Classifier Estimator
* 5-fold cross validation
* Use accuracy to score the models
* Use 4 cores for processing in parallel
* Ensure you refit the best model and return training scores
* Randomly sample 10 models

The hyperparameter grid should be for `learning_rate` (150 values between 0.1 and 2) and `min_samples_leaf` (all values between 20 and 65).

In [ ]:
# Create the parameter grid
param_grid = {'learning_rate': np.linspace(0.1, 2, 150), 'min_samples_leaf': list(range(20, 65))} 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Create a random search object
random_GBM_class = RandomizedSearchCV(
    estimator = GradientBoostingClassifier(random_state=42),
    param_distributions = param_grid,
    n_iter = 10,
    scoring='accuracy', n_jobs=4, cv = 5, refit=True, return_train_score = True)

In [ ]:
# Fit to the training data
random_GBM_class.fit(X_train, y_train)

In [ ]:
# Print the values used for both hyperparameters
print(random_GBM_class.cv_results_['param_learning_rate'])
print(random_GBM_class.cv_results_['param_min_samples_leaf'])

### RandomSearchCV in Scikit Learn

Let's practice building a RandomizedSearchCV object using Scikit Learn.

The desired options are:

* A RandomForestClassifier Estimator with default 80 estimators
* 3-fold cross validation
* Use AUC to score the models
* Use 4 cores for processing in parallel
* Ensure you refit the best model and return training scores
* Randomly sample 5 models for processing efficiency

The hyperparameter grid should be for `max_depth` (all values between and including 5 and 25) and `max_features` ('auto' and 'sqrt').

In [ ]:
# Create the parameter grid
param_grid = {'max_depth': list(range(5,26)), 'max_features': ['auto' , 'sqrt']} 

In [ ]:
# Create a random search object
random_rf_class = RandomizedSearchCV(
    estimator = RandomForestClassifier(n_estimators=80, random_state=42),
    param_distributions = param_grid, n_iter = 5,
    scoring='roc_auc', n_jobs=4, cv = 3, refit=True, return_train_score = True)

In [ ]:
# Fit to the training data
random_rf_class.fit(X_train, y_train)

In [ ]:
# Print the values used for both hyperparameters
print(random_rf_class.cv_results_['param_max_depth'])
print(random_rf_class.cv_results_['param_max_features'])

## Comparing Grid and Random Search

### Grid and Random Search Side by Side

Visualizing the search space of random and grid search together allows us to easily see the coverage that each technique has and therefore brings to life their specific advantages and disadvantages.

In this example, we will sample hyperparameter combinations in a grid search way as well as a random search way, then plot these to see the difference.

In [ ]:
# Sample grid coordinates
grid_combinations_chosen = combinations_list[0:300]

In [ ]:
# Print result
grid_combinations_chosen

In [ ]:
# Create a list of sample indexes
sample_indexes = list(range(0,len(combinations_list)))

In [ ]:
# Randomly sample 300 indexes
random_indexes = np.random.choice(sample_indexes, 300, replace=False)

In [ ]:
# Use indexes to create random sample
random_combinations_chosen = [combinations_list[index] for index in random_indexes]

In [ ]:
def visualize_search(grid_combinations_chosen, random_combinations_chosen):
    grid_y, grid_x = [x[0] for x in grid_combinations_chosen], [x[1] for x in grid_combinations_chosen]
    rand_y, rand_x = [x[0] for x in random_combinations_chosen], [x[1] for x in random_combinations_chosen]

    # Plot all together
    plt.scatter(grid_y + rand_y, grid_x + rand_x, c=['red']*300 + ['blue']*300)
    plt.gca().set(xlabel='learn_rate', ylabel='min_samples_leaf', title='Grid and Random Search Hyperparameters')
    plt.gca().set_xlim([0.01, 3.0])
    plt.gca().set_ylim([5, 24])
    plt.show()

In [ ]:
# Call the function to produce the visualization
visualize_search(grid_combinations_chosen, random_combinations_chosen)

We can really see how a grid search will cover a small area completely whilst random search will cover a much larger area but not completely.

---
**[Week 5 - Data Preprocessing and Hyperparameter Tuning](https://radu-enuca.gitbook.io/ml-challenge/preprocessing-and-tuning)**

*Have questions or comments? Visit the ML Challenge Mattermost Channel.*